### Analyze Emotions

In [3]:
from textblob import TextBlob
from transformers import pipeline
from newsplease import NewsPlease
import re

In [4]:
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
classification = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

c:\Users\marco\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
config.json: 100%|██████████| 1.15k/1.15k [00:00<00:00, 1.21MB/s]
c:\Users\marco\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\marco\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-t

ValueError: Could not load model facebook/bart-large-mnli with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSequenceClassification'>, <class 'transformers.models.bart.modeling_bart.BartForSequenceClassification'>). See the original errors:

while loading with AutoModelForSequenceClassification, an error is thrown:
Traceback (most recent call last):
  File "c:\Users\marco\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\pipelines\base.py", line 269, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\marco\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\auto\auto_factory.py", line 566, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\marco\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\modeling_

In [ ]:
article = NewsPlease.from_url("https://www.npr.org/2023/12/22/1221230635/japan-alleged-political-corruption-ldp-slush-fund")

In [ ]:
article = article.maintext

In [ ]:
# Splitting the text into sentences
sentences = re.split(r'(?<=[.!?])\s+', article)

# Creating a dictionary with the number of characters, words, and the content of each sentence
sentence_dict = []
for i, sentence in enumerate(sentences, 1):
    words = sentence.split()
    dict_append = {
        'chars': len(sentence), 
        'words': len(words), 
        'content': sentence,
        'emotions': {}
    }
    for item in emotion_classifier(sentence)[0]:
        dict_append['emotions'][item['label']] = round(item['score'] * 100, 2)

    sentence_dict.append(dict_append)

In [ ]:
emotion_sum_char = 0
emotion_value = 0
emotions_percentage = {}

for emotion in ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']:
    for sentence in sentence_dict:
        emotion_value += sentence['chars'] * sentence['emotions'][emotion]
        emotion_sum_char += sentence['chars']
    emotions_percentage[emotion] = emotion_value / emotion_sum_char
    emotion_value = 0
    emotion_sum_char = 0

In [ ]:
emotions_percentage

{'anger': 12.281852131271219,
 'disgust': 24.190711052433038,
 'fear': 4.047276499434177,
 'joy': 1.4526574877404757,
 'neutral': 49.68121463598643,
 'sadness': 7.135081101471144,
 'surprise': 1.213379856657865}

In [ ]:
sum_emotions = 0
for i in emotions_percentage.values():
    sum_emotions += i
sum_emotions

100.00217276499434

In [ ]:
blob = TextBlob(article)

blob.sentiment

Sentiment(polarity=0.013236607142857139, subjectivity=0.3929129464285715)

In [5]:
candidate_labels = ["Elections",
            "International Relations",
            "Policy Reforms",
            "Legislation",
            "Civil Right",
            "Defense And Security",
            "Local Governance",
            "Politics Scandals",
            "Public Opinion",
            "Political Parties",
            "Global Economy",
            "Stock Market",
            "Banking",
            "Real Estate",
            "Cryptocurrencies",
            "Personal Finance",
            "Insurance",
            "Taxation",
            "Corporate Finance",
            "Economic Policies",
            "Climate Change",
            "Renewable Energy",
            "Wildlife",
            "Pollution",
            "Sustainable Living",
            "Natural Disasters",
            "Agriculture",
            "Water Resources",
            "Environment Laws",
            "Biodiversity"]
classifier(article, candidate_labels)

NameError: name 'classifier' is not defined

## Download models

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [15]:
model_name = 'mistralai/Mixtral-8x7B-v0.1'

# Download and cache the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

  warnings.warn(
c:\Users\marco\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:983: UserWarning: Not enough free disk space to download the file. The expected file size is: 4899.04 MB. The target location C:\Users\marco\.cache\huggingface\hub\models--mistralai--Mixtral-8x7B-v0.1\blobs only has 2471.10 MB free disk space.
  warnings.warn(
model-00004-of-00019.safetensors:  40%|████      | 1.98G/4.90G [01:53<02:46, 17.5MB/s]


KeyboardInterrupt: 

In [ ]:
# Save the model and tokenizer to a directory
model.save_pretrained('./models/Mixtral-8x7B-v0.1_model/')
tokenizer.save_pretrained('./models/Mixtral-8x7B-v0.1_model/')

In [ ]:
# Load the model and tokenizer from the directory
model = AutoModelForCausalLM.from_pretrained('./models/Mixtral-8x7B-v0.1_model/')
tokenizer = AutoTokenizer.from_pretrained('./models/Mixtral-8x7B-v0.1_model/')

In [ ]:
# Function to generate a response
def generate_response(model, tokenizer, chat_history_ids, text):
    # Encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(text + tokenizer.eos_token, return_tensors='pt')

    # Append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if chat_history_ids is not None else new_user_input_ids

    # Generate a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # Retrieve the last response
    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response, chat_history_ids

# Initialize chat history
chat_history_ids = None

# Example of chatting
user_input = "Hello, how are you?"
response, chat_history_ids = generate_response(model, tokenizer, chat_history_ids, user_input)
print("Bot:", response)